# Testing the GPU performance on MNIST

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import argparse
import time

# Downloading and Processing MNIST Data

In [ ]:
train, test = mnist.load_data()
Xtrain, Ytrain = train
Xtest, Ytest = test
# MNIST Dataset image is 28x28x1
# For Feed Forward Neural Network, flatten the data
Xtrain = Xtrain.reshape((Xtrain.shape[0], 28 * 28 * 1))
Xtest = Xtest.reshape((Xtest.shape[0], 28 * 28 * 1))
# Normalize data in the range [0, 1]
Xtrain = Xtrain.astype("float32") / 255.0
Xtest = Xtest.astype("float32") / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
Ytrain = lb.fit_transform(Ytrain)
Ytest = lb.transform(Ytest)

# Model Architecture

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))

# Training

In [ ]:
model.compile(
    loss="categorical_crossentropy", optimizer=SGD(0.01), metrics=["accuracy"]
)

print("Begin Training")
start_time = time.time()
H = model.fit(
    Xtrain, Ytrain, validation_data=(Xtest, Ytest), epochs=100, batch_size=128
)
end_time = time.time()
print("Total Time Taken: {}".format(end_time - start_time))
print("Average Time Per Epoch: {}".format((end_time - start_time) / 100))


# Evaluation

In [ ]:
predictions = model.predict(Xtest, batch_size=128)
print(
    classification_report(
        Ytest.argmax(axis=1),
        predictions.argmax(axis=1),
        target_names=[str(x) for x in lb.classes_],
    )
)

# Plot and save the graph

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("output.png")